In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from tf.keras.layers import Conv2D

ModuleNotFoundError: No module named 'tf'

# Let's build GoogleNet 


## This architecture uses the Inception module, We will explain this module :)


<h3>Important notes about the first reading : </h3>
<br>
<ul> 
    <li>
        <i>" 1 × 1 convolutions have dual purpose: most critically, they are used mainly as dimension reduction modules to remove computational bottlenecks "</i>
    </li>
    <li>
        <i>The most straightforward way of improving the performance of deep neural networks is by increasing their size</i>
    </li>
    <li>
        <i>Bigger size typically means a larger number of parameters, which makes the enlarged network more
            prone to overfitting</i>
    </li>
    <li>
        <i>Another drawback of uniformly increased network size is the dramatically increased use of computational resources</i>
    </li>
    <li>
        <i> current incarnations of the Inception architecture are restricted to filter sizes 1×1,
3×3 and 5×5, however this decision was based more on convenience rather than necessity</i>
    <li>
        <img src="inception.PNG"/>
    </li>
    <li>
        <i>Besides being used as reductions, they also include the use of rectified linear activation which makes them dual-purpose</i>
    </li>
    <li>
        <i>occasional max-pooling layers with stride 2 to halve the resolution of the grid</i>
    </li>
    <li>
        <img src="GoogleNetSummary.PNG">
    </li>
    <li>
        <i>
            We have also
used a deeper and wider Inception network, the quality of which was slightly inferior, but adding it
to the ensemble seemed to improve the results marginally.
        </i>
    </li>
</ul>

<h2>Topology</h2>

<ul>
    <li><i>size of the receptive field in our network is 224×224 taking RGB color channels with mean subtraction</i></li>
    <li><i>“#3×3 reduce” and “#5×5 reduce” stands for the number of 1×1 filters in the reduction</i></li>
    <li><i>The network is 22 layers deep when counting only layers with
        parameters (or 27 layers if we also count pooling)</i></li>
    <li><i>The overall number of layers (independent building blocks) used for the construction of the network is about 100.</i></li>
    <li><i>however the use of dropout remained
        essential even after removing the fully connected layers</i></li>
    <li><i>By adding auxiliary classifiers connected to
these intermediate layers, we would expect to encourage discrimination in the lower stages in the
classifier, increase the gradient signal that gets propagated back, and provide additional regularization</i></li>
    <li>First extra Classifier : </li>
    <img src="first_classifier.PNG">
    <li>Second extra classifier </li>
    <img src="extra_classifier2.PNG">
    <li><i>An average pooling layer with 5×5 filter size and stride 3, resulting in an 4×4×512 output
        for the (4a), and 4×4×528 for the (4d) stage</i></li>
    <li><i>A 1×1 convolution with 128 filters for dimension reduction and rectified linear activation.</i></li>
    <li><i>A fully connected layer with 1024 units and rectified linear activation.</i></li>
    <li><i>A dropout layer with 70% ratio of dropped outputs</i></li>
    <li><i>A linear layer with softmax loss as the classifier (predicting the same 1000 classes as the
        main classifier, but removed at inference time).</i></li>
    <img src="full_GoogleNet.PNG">
</ul>
<h2>Training </h2>

<ul>
    <li><i> Specifically, we resize the image to 4 scales where the shorter dimension (height or
        width) is 256, 288, 320 and 352 respectively</i></li>
    <li><i>During training, their loss gets added to the total loss of the
network with a discount weight (the losses of the auxiliary classifiers were weighted by 0.3). At
        inference time, these auxiliary networks are discarded.</i></li>

## The Inception block

In [ ]:

#Inception block: 
    #X1 1x1x64 s1  
    #X2 1x1x96 s1 => # 3x3x128 s1 : []
    #X3 1x1x16 s1 => 5x5x32 s1 
    #X4 3x3x192 s1 Max Pool => 1x1x32 s1 
    #X1+X2+X3+X4 Concat

def inception_block(conv1=[64], conv2=[96,128], conv3=[16,32], conv4=[192, 32], classifier=False): 
    